In [ ]:
plot_samples = False
import torchvision.datasets.mnist
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim

In [ ]:
transform = transforms.ToTensor()

# load the mnist dataset, training and testsplit
mnist_train = torchvision.datasets.MNIST(root='/content/data', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='/content/data', train=False, download=True, transform=transform)

# plot an image from the training and testing set
train_sample = mnist_train[0]
test_sample = mnist_test[0]
if plot_samples:
    plt.imshow(train_sample[0].squeeze(), cmap="grey")
    plt.savefig("train_sample")
    plt.imshow(test_sample[0].squeeze(), cmap="grey")
    plt.savefig("test_sample")

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 16029820.25it/s]


Extracting /content/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 495990.23it/s]


Extracting /content/data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:00<00:00, 4428749.80it/s]


Extracting /content/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 3316596.23it/s]


Extracting /content/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/MNIST/raw



In [ ]:
train_dataloader = DataLoader(mnist_train)
test_dataloader = DataLoader(mnist_test)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([1, 1, 28, 28])
Shape of y: torch.Size([1]) torch.int64


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, topology):
        super().__init__()
        self.flatten = nn.Flatten()
        match topology:
            case "single_hidden_layer":
                self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )
            case "two_layers":
                self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 500),
                nn.ReLU(),
                nn.Linear(500, 300),
                nn.ReLU(),
                nn.Linear(300, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )
            case "cnn":
              self.conv_layers = nn.Sequential(
                nn.Conv2d(1, 32, kernel_size=3, padding=1),  # First convolutional layer
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),  # Reduces image to 14x14

                nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Second convolutional layer
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2)  # Reduces image to 7x7
              )
              self.fc_layers = nn.Sequential(
                nn.Flatten(),
                nn.Linear(64 * 7 * 7, 512),  # Fully connected layer
                nn.ReLU(),
                nn.Linear(512, 10)  # Output layer for 10 classes
              )

    def forward(self, x):
        x = self.conv_layers(x)  # Apply convolutional layers
        x = self.fc_layers(x)     # Apply fully connected layers
        return x

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model = NeuralNetwork(topology="two_layers")
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Streaming output truncated to the last 5000 lines.
loss: 0.003656  [44401/60000]
loss: 0.029567  [44501/60000]
loss: 0.027935  [44601/60000]
loss: 0.000614  [44701/60000]
loss: 0.051096  [44801/60000]
loss: 0.000400  [44901/60000]
loss: 0.006519  [45001/60000]
loss: 0.013125  [45101/60000]
loss: 0.000491  [45201/60000]
loss: 0.260694  [45301/60000]
loss: 0.002092  [45401/60000]
loss: 0.148377  [45501/60000]
loss: 0.049986  [45601/60000]
loss: 0.159666  [45701/60000]
loss: 0.777169  [45801/60000]
loss: 0.020397  [45901/60000]
loss: 0.020436  [46001/60000]
loss: 0.117198  [46101/60000]
loss: 0.053084  [46201/60000]
loss: 2.514192  [46301/60000]
loss: 0.003631  [46401/60000]
loss: 0.000605  [46501/60000]
loss: 0.461235  [46601/60000]
loss: 0.001705  [46701/60000]
loss: 0.037060  [46801/60000]
loss: 0.000932  [46901/60000]
loss: 0.007427  [47001/60000]
loss: 0.542964  [47101/60000]
loss: 0.091043  [47201/60000]
loss: 0.088301  [47301/60000]
loss: 0.013891  [47401/60000]
loss: 0.002722  [47

# Problem 4

In [ ]:
import torch.optim as optim
epochs = 40

model = NeuralNetwork(topology="cnn")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-4)  # L2 regularization with weight_decay

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

KeyboardInterrupt: 